In [1]:
import pandas as pd
import numpy as np
from glob import glob


In [2]:

file_lst = glob("raw/*.csv")


In [3]:
class RES:
    def __init__(self, features=[]):
        self.ft_n = len(features)
        self.features=features
        self.RES = np.empty((1, self.ft_n))
        self.RES_len=0
        self.RES_cap=1

    def add(self,x):
        if isinstance(x, pd.DataFrame):
            x = x.to_numpy()        
        size = x.shape[0]
        if (size + self.RES_len > self.RES_cap):
            while (size + self.RES_len > self.RES_cap):
                self.RES_cap *= 2
            
            newRes = np.empty((self.RES_cap, self.ft_n))
            newRes[0:self.RES_len,:] = self.RES[0:self.RES_len,:]
            self.RES = newRes
        self.RES[self.RES_len:self.RES_len+size, :] = x
        self.RES_len += size

    def getDf(self):
        return pd.DataFrame(self.RES[0:self.RES_len,:], columns=self.features)

    def __len__(self):
        return self.RES_len
        
    def __str__(self):
        return self.RES.__str__()


In [4]:

n_normal = 0
n_attack = 0
req_ft=['duration','src_bytes','dst_bytes','missed_bytes','src_pkts','dst_pkts','label']
print(len(req_ft))
r = RES(req_ft)
for f in file_lst:    
    df = pd.read_csv(f, low_memory=False)  
    df = df[df['type'].isin(['normal', 'dos', 'ddos', 'scanning'])]
    df = df.loc[:, df.columns.isin(req_ft)]
    size = df.shape[0]
    
    at_df = df[df['label'] == 1]
    nr_df = df[df['label'] == 0]
    n_nr = nr_df.shape[0]
    n_normal += n_nr
    n_attack += at_df.shape[0]    
    print(len(r))    
    
    rng = np.random.default_rng()
    r.add(rng.choice(df.to_numpy(), size=max(0,int(size*0.1)), replace=False))
    del df
    


7
0
100000
200000
300000
301440
388920
488920
491028
591028
691028
758281
858281
958281
1058281
1158281
1258281
1358281
1381779
1443066
1543066
1643066
1643558


In [5]:
print (f"NORMAL: {n_normal} ATTACK: {n_attack}")

NORMAL: 755124 ATTACK: 16680497


In [6]:
df = r.getDf()
df.to_csv("processed/imbalanced_dataset.csv", index=False)

In [7]:


df = pd.read_csv("processed/imbalanced_dataset.csv")
print(df[df.isnull().any(axis=1)].shape[0])
N=df.shape[0]
train_size=int((N*3)/4)
print(f"N: {N}, TS:{train_size}")
test_df = df.loc[train_size:N]
df = df.loc[0:train_size]

0
N: 1743558, TS:1307668


In [8]:
test_df.to_csv('processed/test_data.csv', index=False)

In [13]:
# from imblearn.over_sampling import ADASYN
# Y = df['label'].to_numpy().astype(np.uint8)
# X = df.drop('label', axis=1).to_numpy()
# df[df['label'].isnull()].head()
# print(Y[0:100])



In [14]:
# from imblearn.over_sampling import ADASYN
# X_r, Y_r = ADASYN().fit_resample(X, Y)
# res=np.empty((X_r.shape[0], df.shape[1]))
# res[:, 0:-1] = X_r
# res[:, -1] = Y_r
# pd.DataFrame(res, columns = df.columns).to_csv("processed/balanced_dataset.csv", index=False)

In [15]:
# np.bincount(Y_r)

In [16]:
# np.random.shuffle(res)


In [9]:
df.to_csv("processed/train_data.csv", index=False)

In [10]:
df.tail(15)

,duration,src_bytes,dst_bytes,missed_bytes,src_pkts,dst_pkts,label
1307654,63.243876,0.0,0.0,0.0,3.0,2.0,1.0
1307655,61.560365,438.0,1791.0,0.0,7.0,15.0,1.0
1307656,0.000632,0.0,0.0,0.0,2.0,1.0,1.0
1307657,0.000005,0.0,0.0,0.0,2.0,1.0,1.0
1307658,0.000476,0.0,0.0,0.0,2.0,1.0,1.0
1307659,0.024844,121.0,171.0,0.0,6.0,4.0,1.0
1307660,0.000004,0.0,0.0,0.0,2.0,1.0,1.0
1307661,0.004301,39.0,39.0,0.0,1.0,1.0,1.0
1307662,0.003039,43.0,43.0,0.0,1.0,1.0,1.0
1307663,60.795875,0.0,0.0,0.0,3.0,2.0,1.0
